### https://github.com/HarmanDotpy/Named-Entity-Recognition-in-Pytorch/blob/main/scripts/train_bilstm_char_random_glove.py

In [1]:
import torch
torch.manual_seed(10)
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence, pad_sequence
from torch.utils.data import Dataset, DataLoader, TensorDataset
import io
import sklearn
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import pickle as pickle
import warnings
from sklearn.exceptions import UndefinedMetricWarning
import seqeval
from seqeval.metrics import accuracy_score as seq_accuracy_score
from seqeval.metrics import classification_report as seq_classification_report
from seqeval.metrics import f1_score as seq_f1_score
import pandas as pd

In [2]:
## LSTM for CHARACTER level

class forLSTM(nn.Module):
    def __init__(self, embedding_size, hidden_size, pretr_char_embed):
        super(forLSTM, self).__init__()
        self.charembed = nn.Embedding.from_pretrained(pretr_char_embed, freeze = False) #size of pretrained = (totalchars,embedding size)
        self.lstm = nn.LSTM(embedding_size, hidden_size, bidirectional = True, batch_first = True)

    def forward(self, xchar, xlength_char):
        #xchar is of shape(batchsize, seqlen_maxinbatch, maxwordlen-ie max char = 6)

        shape = xchar.shape
        xchar = xchar.view(-1, shape[2])
        xlength_char = xlength_char.view(-1)
        input = pack_padded_sequence(xchar, xlength_char.cpu(), batch_first=True, enforce_sorted=False)
        input, _ = pad_packed_sequence(input, batch_first=True)
        embed = self.charembed(input)
        _, (h,_) = self.lstm(embed) #h is of size (2, 128*maxno. of words in a sentence in the batch, 25)
        h = h.view(h.shape[1], 50)
        h = h.view(shape[0], shape[1], 50)
        return h

In [3]:
## BILSTM model

class BiLSTM(nn.Module):
    def __init__(self, embedding_size, hidden_size, total_words, num_class, pretrained = False, pretrained_embed = None, char_embed_size = 0, pretr_char_embed = None):
        super(BiLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.wordembed = nn.Embedding.from_pretrained(pretrained_embed, freeze = False)
        self.for_charembed = forLSTM(embedding_size = char_embed_size, hidden_size = 25, pretr_char_embed = pretr_char_embed)
        self.dropout = nn.Dropout(p = 0.5)
        self.bilstm = nn.LSTM(embedding_size + 50,hidden_size, bidirectional = True, batch_first = True)
        self.linear = nn.Linear(2*hidden_size, num_class) # 2 because forward and backward concatenate

    def forward(self, x, xchar, xlengths, xlength_char):
        x = pack_padded_sequence(x, xlengths.cpu(), batch_first=True, enforce_sorted=False)
        x, _ = pad_packed_sequence(x, batch_first=True)

        xlength_char = pack_padded_sequence(xlength_char, xlengths.cpu(), batch_first=True, enforce_sorted=False)
        xlength_char, _ = pad_packed_sequence(xlength_char, batch_first=True, padding_value = len("<pad>")) 
        # above this line padding value is taken as len of pad word becasue that is what we pad sentences 
        # with hance at a character level it should be the length

        xchar = pack_padded_sequence(xchar, xlengths.cpu(), batch_first=True, enforce_sorted=False)
        xchar, _ = pad_packed_sequence(xchar, batch_first=True)

        word_embedding = self.wordembed(x) # x is of size(batchsize, seq_len), wordembed is of size (batchsize, seq_len, embedding_size = 100)
        forwardchar= self.for_charembed(xchar, xlength_char) #forwardchar and backwardchar would be of size (batchsize, seqlen. embedding_size = 25each) 
        word_embedding = torch.cat((word_embedding, forwardchar), dim = 2)

        word_embedding = self.dropout(word_embedding) #dropout
        out, (h,c) = self.bilstm(word_embedding) #'out' has dimension(batchsize, seq_len, 2*hidden_size)

        out = self.linear(out) #now 'out' has dimension(batchsize, seq_len, num_class)
        out = out.view(-1, out.shape[2]) # shape (128*seqlen, 18)
        out = F.log_softmax(out, dim=1) # take the softmax across the dimension num_class, 'out' has dimension(batchsize, seq_len, num_class)
        return out

#### HELPER FUNCTIONS

In [4]:
# reading text file in python and making list of sentences (list of lists) and list of tags(list of lists)
def load_data(datapath, buildvocab_tags= True, vocab = None, nertags = None):
    if(buildvocab_tags == True):
        all_words = []
        all_tags = []
        with open(datapath) as f:
            lines = f.readlines()
            sent_num = 0
            for line in lines[2:]: 
                if(line == "\n"):
                    sent_num+=1
                else:
                    line_sep = line.split(sep = " ")
                    all_words.append(line_sep[0])
                    all_tags.append(line_sep[3][:-1])
                    
        words = list(set(all_words))
        tags = list(set(all_tags))

        vocab = {}
        vocab['<pad>'] = 0 # for padding input sequences
        vocab['<oov>'] = 1
        for i, word in enumerate(words):
            vocab[word] = i+2
            
        nertags = {}
        nertags['padtag'] = 0
        for i,nertag in enumerate(tags):
            nertags[nertag] = i+1

    train_sent = []
    train_tags = []
    with open(datapath) as f:
        lines = f.readlines()
        sent_num = 0
        sentence = []
        tag = []
        for line in lines[2:]: #1: so that the first blank line isn't taken into account
            if(line == "\n"):
                sent_num+=1
                train_sent.append(sentence)
                train_tags.append(tag)
                sentence = []
                tag = []
            else:
                line_sep = line.split(sep = " ")
                if(line_sep[0] in vocab.keys()):
                    sentence.append(vocab[line_sep[0]])
                else:
                    sentence.append(vocab['<oov>'])
                    
                tag.append(nertags[line_sep[3][:-1]])

    # padding the sentences at the end
    seq_maxlen = max(len(x) for x in train_sent)
    x_lengths = [len(x) for x in train_sent]
    Xtrain = []
    Ytrain = []
    for sent, tags in zip(train_sent, train_tags):
        length_toappend = seq_maxlen - len(sent)
        Xtrain.append(sent+[0]*length_toappend)
        Ytrain.append(tags+[0]*length_toappend)


    Xtrain = torch.Tensor(Xtrain)
    Ytrain = torch.Tensor(Ytrain)
    x_lengths = torch.Tensor(x_lengths)
    print(Xtrain.shape, Ytrain.shape, x_lengths.shape)
    
    return Xtrain, Ytrain, x_lengths, vocab, nertags


In [5]:
def test_load_data(datapath, buildvocab_tags= True, vocab = None, nertags = None):
    if(buildvocab_tags == True):
        all_words = []
        all_tags = []
        with open(datapath) as f:
            lines = f.readlines()
            sent_num = 0
            for line in lines: #1: so that the first blank line isn't taken into account
                if(line == "\n"):
                    sent_num+=1
                else:
                    line_sep = line.split(sep = " ")
                    all_words.append(line_sep[0])
                    all_tags.append(line_sep[1])
                    
        words = list(set(all_words))
        tags = list(set(all_tags))

        vocab = {}
        vocab['<pad>'] = 0 # for padding input sequences
        vocab['<oov>'] = 1
        for i, word in enumerate(words):
            vocab[word] = i+2
            
        nertags = {}
        nertags['padtag'] = 0
        for i,nertag in enumerate(tags):
            nertags[nertag] = i+1

    train_sent = []
    train_tags = []
    with open(datapath) as f:
        lines = f.readlines()
        sent_num = 0
        sentence = []
        tag = []
        for line in lines: #1: so that the first blank line isn't taken into account
            if(line == "\n"):
                sent_num+=1
                train_sent.append(sentence)
                train_tags.append(tag)
                sentence = []
                tag = []
            else:
                line_sep = line.split(sep = " ")
                if(line_sep[0] in vocab.keys()):
                    sentence.append(vocab[line_sep[0]])
                else:
                    sentence.append(vocab['<oov>'])
                    
                tag.append(nertags[line_sep[1]])

    # padding the sentences at the end
    seq_maxlen = max(len(x) for x in train_sent)
    x_lengths = [len(x) for x in train_sent]
    Xtrain = []
    Ytrain = []
    for sent, tags in zip(train_sent, train_tags):
        length_toappend = seq_maxlen - len(sent)
        Xtrain.append(sent+[0]*length_toappend)
        Ytrain.append(tags+[0]*length_toappend)


    Xtrain = torch.Tensor(Xtrain)
    Ytrain = torch.Tensor(Ytrain)
    x_lengths = torch.Tensor(x_lengths)
    #print(Xtrain.shape, Ytrain.shape, x_lengths.shape)
    
    return Xtrain, Ytrain, x_lengths, vocab, nertags


In [6]:
def load_char_data(words, charvocab):
    train_char_sent = []
    train_char_label = []
    for word in words:
        chars = []
        char_labels = []

        word_sep = list(word)
        for c in word_sep[:-1]:
            if (c in charvocab.keys()):
                chars.append(charvocab[c])
            else:
                chars.append(charvocab['<oovchar>'])
        for c in word_sep[1:]:
            if (c in charvocab.keys()):
                char_labels.append(charvocab[c])
            else:
                char_labels.append(charvocab['<oovchar>'])
        
        train_char_sent.append(chars)
        train_char_label.append(char_labels)

    # padding the char_sents at the end
    seq_maxlen = max(len(x) for x in train_char_sent)
    x_lengths_char = [len(x) for x in train_char_sent]
    Xtrain_char = []
    Ytrain_char = []
    for char_sent, char_label in zip(train_char_sent, train_char_label):
        length_toappend = seq_maxlen - len(char_sent)
        Xtrain_char.append(char_sent+[0]*length_toappend)
        Ytrain_char.append(char_label+[0]*length_toappend) # 0 is padchar


    Xtrain_char = torch.Tensor(Xtrain_char)
    Ytrain_char = torch.Tensor(Ytrain_char)
    x_lengths_char = torch.Tensor(x_lengths_char)
    # print(Xtrain.shape, Ytrain.shape, x_lengths.shape)
    
    return Xtrain_char, Ytrain_char, x_lengths_char

def pad_chars(topadlist, maxlen):
    topadlist = topadlist + [0]*(maxlen-len(topadlist))

    return topadlist

def make_id2word_charvocab(vocab, charvocab):
    max_charlen = max(len(word) for word in vocab.keys())
    word_charlevel_vocab = {}
    wordid2wordlen = {}
    for word in vocab.keys():
        word_charlevel_vocab[vocab[word]] = [charvocab[w] if w in charvocab.keys() else charvocab['<oovchar>'] for w in word]
        word_charlevel_vocab[vocab[word]] = pad_chars(word_charlevel_vocab[vocab[word]], max_charlen)

        wordid2wordlen[vocab[word]] = len(word)
        # word_charlevel_vocab[vocab[word]] = word_charlevel_vocab[vocab[word]].extend([charvocab['<padchar>']]*(max_charlen-len(word_charlevel_vocab[vocab[word]])))
    return word_charlevel_vocab, wordid2wordlen


def load_char_level(X, wordid2word_charlevel_vocab, wordid2wordlen):
    #X is of shape (no.of.sentences, 104)
    Xcharlevel = [] # will finally be fo shape (total.sentences, max_sent.len, )
    Xcharlevel_lengths = []
    for i in range(X.shape[0]):
        sentence = []
        wordlengths = []
        for j in range(X.shape[1]):
            sentence.append(torch.tensor([wordid2word_charlevel_vocab[int(X[i, j].item())]]))
            wordlengths.append(wordid2wordlen[int(X[i, j].item())])
            # sentences = pad_sequence(sentences)
        # print(i)
        Xcharlevel_lengths.append(wordlengths)
        Xcharlevel.append(torch.stack(sentence))
    
    return torch.squeeze(torch.stack(Xcharlevel)), torch.tensor(Xcharlevel_lengths)

def get_charvocab(vocab):
    # using vocab make charvocab
    words = list(vocab.keys())
    characters = [char for word in words for char in word]
    characters = list(set(characters))
    char_vocab = {}
    char_vocab["<padchar>"] = 0
    char_vocab["<oovchar>"] = 1
    for i, char in enumerate(characters):
        char_vocab[char] = i+2

    return char_vocab


#### Loading data

In [7]:
traindatapath = 'data/train.txt'
devdatapath = 'data/dev.txt'
testdatapath = 'data/test.txt'

Xtrain, Ytrain, x_trainlengths, vocab, nertags = load_data(traindatapath, buildvocab_tags=True)
Xdev, Ydev, x_devlengths, _, _ = load_data(devdatapath, buildvocab_tags=False, vocab = vocab, nertags = nertags)

torch.Size([14986, 113]) torch.Size([14986, 113]) torch.Size([14986])
torch.Size([3465, 109]) torch.Size([3465, 109]) torch.Size([3465])


In [8]:
# Character Level training data making
# make vocabulary of characters from train vocabulary
char_vocab = get_charvocab(vocab)
wordid2word_charlevel_vocab, wordid2wordlen = make_id2word_charvocab(vocab, char_vocab) # of the form {word:[1,2,3,4]}, {wordnum:wordlen}
#make char level train data for the char embeddings 
Xtrain_char, xlength_char = load_char_level(Xtrain, wordid2word_charlevel_vocab, wordid2wordlen)
#finally make the dataloader for train
traindataset = TensorDataset(Xtrain, Xtrain_char, Ytrain, x_trainlengths, xlength_char)
Trainloader = DataLoader(traindataset, batch_size= 128, shuffle=True)

# Character Level validation data making
Xdev_temp, Ydev_temp, x_devlengths_temp, devvocab, devnertags = load_data(devdatapath, buildvocab_tags=True)
wordid2word_charlevel_vocab_dev, wordid2wordlen_dev = make_id2word_charvocab(devvocab, char_vocab) # of the form {word:[1,2,3,4]}, {wordnum:wordlen}
#make char level train data for the char embeddings 
Xdev_char, xdevlength_char = load_char_level(Xdev_temp, wordid2word_charlevel_vocab_dev, wordid2wordlen_dev)
#finally make the dataloader for train
devdataset = TensorDataset(Xdev, Xdev_char, Ydev, x_devlengths, xdevlength_char)
Devloader = DataLoader(devdataset, batch_size= 128, shuffle=True)

torch.Size([3465, 109]) torch.Size([3465, 109]) torch.Size([3465])


In [54]:
# DEFINE MY MODEL!!! 

pre_embeddings = 'glove'
Expname = 'BILSTM_glove_char'
rootpath = 'out/'
glove_embeddings_file = 'data/glove.6B.100d.txt'

In [9]:
if torch.cuda.is_available():  
    device = "cuda:0" 
else:  
    device = "cpu"  

In [10]:
# LOAD EMBEDDINGS
embedding_size = 100

num_words = len(vocab)
word_embeds = torch.rand(num_words, embedding_size)

In [11]:
# character level onehot embeddings and important classes for performance metrics
char_onehot = torch.eye(len(char_vocab))
imp_classes = [nertags[tag] for tag in nertags.keys()]
imp_classes.remove(nertags['padtag'])
imp_classes.remove(nertags['O'])

In [12]:
model = BiLSTM(embedding_size = 100, hidden_size = 100, total_words = len(vocab), num_class = len(nertags), pretrained = True, pretrained_embed = word_embeds, char_embed_size = len(char_vocab), pretr_char_embed = char_onehot).to(device) 
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3) 
lossfunction = nn.CrossEntropyLoss()

In [13]:
print(model)

BiLSTM(
  (wordembed): Embedding(23626, 100)
  (for_charembed): forLSTM(
    (charembed): Embedding(88, 88)
    (lstm): LSTM(88, 25, batch_first=True, bidirectional=True)
  )
  (dropout): Dropout(p=0.5, inplace=False)
  (bilstm): LSTM(150, 100, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=200, out_features=10, bias=True)
)


In [14]:
def performance(y, ypred, nertags):
    y = y.numpy()
    ypred = ypred.numpy()
    mask = (y != nertags['padtag']) * (y != nertags['O'])
    y = y*mask
    ypred = ypred*mask
    acc = ((y==ypred)*mask).sum()/mask.sum()
    microf1 = f1_score(y, ypred, labels = imp_classes, average='micro')
    macrof1 = f1_score(y, ypred, labels = imp_classes, average='macro')

    return acc, microf1, macrof1

In [15]:
def validate(model, loader):
        with torch.no_grad():
            validloss = 0
            acc = 0
            microf1 = 0
            macrof1 = 0
            i = 0
            for step, (X, Xchar, Y, xlen, xlen_char) in enumerate(loader):
                Y = pack_padded_sequence(Y, xlen, batch_first=True, enforce_sorted=False)
                Y, _ = pad_packed_sequence(Y, batch_first=True)
                ypred = model(X.long().to(device), Xchar.to(device), xlen.to(device), xlen_char.to(device))#.permute(0, 2, 1)
                vloss = lossfunction(ypred.to('cpu'), Y.view(-1).type(torch.LongTensor))
                validloss+=vloss.item()
                acc_, microf1_, macrof1_ = performance(Y.view(-1), torch.argmax(ypred.to('cpu'), dim = 1), nertags)
                acc+=acc_
                microf1 += microf1_
                macrof1 += macrof1_
                i+=1

        return validloss/i, acc/i, microf1/i, macrof1/i

In [16]:
trainlosslist = []
trainacclist = [] #accuracy except pad, O
trainmicrof1list = []
trainmacrof1list = []

validlosslist = []
valacclist = []
valmicrof1list = []
valmacrof1list = []

In [17]:
# Model is ready now we have to train using cross entropy loss
num_epochs = 10

warnings.filterwarnings("ignore", category=UndefinedMetricWarning)

# validloss = []
model.train()
for epoch in range(num_epochs):
    if(epoch == 8):
        optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
        
    totalloss, acc, microf1, macrof1 = 0, 0, 0, 0
    for step, (Xbatch , Xchar ,Ybatch, xbatch_len, xlength_char) in enumerate(Trainloader):
        #make gradients 0
        optimizer.zero_grad()

        Ybatch = pack_padded_sequence(Ybatch, xbatch_len, batch_first=True, enforce_sorted=False)
        Ybatch, y_lengths = pad_packed_sequence(Ybatch, batch_first=True)

        #get output from model and claculate loss
        ypred = model(Xbatch.long().to(device), Xchar.to(device), xbatch_len.to(device), xlength_char.to(device))#.permute(0, 2, 1)
        acc_, microf1_, macrof1_ = performance(Ybatch.view(-1), torch.argmax(ypred.to('cpu'), dim = 1), nertags)
        acc+= acc_
        microf1+=microf1_
        macrof1+=macrof1_
        if(step%20 == 0 and step !=0):
            print("training accuracy = {}, microF1 = {}, macroF1 = {}".format(acc/(step+1), microf1/(step+1), macrof1/(step+1)))
            
        loss = lossfunction(ypred.to('cpu'), Ybatch.view(-1).type(torch.LongTensor)) #Ybatch has dimension (batchsize, seqlen), ypred has dimension(batchsize, num_classes, seqlen)
        totalloss += loss.item()

        #backward and step
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), 5) # clip gradient to 5
        optimizer.step()
        
    trainlosslist.append(totalloss/(step+1))
    trainacclist.append(acc/(step+1))
    trainmicrof1list.append(microf1/(step+1))
    trainmacrof1list.append(macrof1/(step+1))

    # model validation loss and scheduler step for learning rate change if required
    val_loss, val_acc, val_microf1, val_macrof1  = validate(model, Devloader)
    validlosslist.append(val_loss)
    valacclist.append(val_acc)
    valmicrof1list.append(val_microf1)
    valmacrof1list.append(val_macrof1)
        
    # scheduler.step(val_loss)
    print('\nepoch = {}, training_loss = {}, validation_loss ={}, training_acc = {}, validation_acc ={}'.format(epoch, trainlosslist[-1], validlosslist[-1], trainacclist[-1], valacclist[-1]))        
    

training accuracy = 0.0027417027417027417, microF1 = 0.0032312925170068026, macroF1 = 0.00224598021221763
training accuracy = 0.0016474002270987825, microF1 = 0.0021338421830090263, macroF1 = 0.0014386705896059207
training accuracy = 0.02029210339038567, microF1 = 0.032904532543002976, macroF1 = 0.018036064082439348
training accuracy = 0.059521659706339984, microF1 = 0.08535963663914753, macroF1 = 0.04471193914365762
training accuracy = 0.10311606815604, microF1 = 0.13696007483454967, macroF1 = 0.07736416024550764

epoch = 0, training_loss = 0.1919276345584352, validation_loss =0.10008564657930817, training_acc = 0.14440225015144809, validation_acc =0.41547443926445077
training accuracy = 0.4931081116911082, microF1 = 0.5294556467133628, macroF1 = 0.4043848144294892
training accuracy = 0.5061359762622567, microF1 = 0.5437206513106206, macroF1 = 0.42210027156909474
training accuracy = 0.5359301035504214, microF1 = 0.5738228045368354, macroF1 = 0.46248145412524766
training accuracy = 0.5

In [18]:
id2word = {id: word for word, id in vocab.items()} 

id2tag = {}
for tag in nertags.keys():
    if(tag == 'padtag'):
         id2tag[nertags[tag]] = 'O' # because we dont want the model to predict 'padtag' tags
    else:
        id2tag[nertags[tag]] = tag

In [15]:
# SAVE MODEL AND DICTIONARIES ---- delete!!

torch.save(model.state_dict(), 'models/bilstm_char/trained_bilstm_model_state_dict.pth')
torch.save(model, 'models/bilstm_char/trained_bilstm_model.pth')

with open('models/bilstm_char/id2word.pkl', 'wb') as f:
    pickle.dump(id2word, f)
with open('models/bilstm_char/id2tag.pkl', 'wb') as f:
    pickle.dump(id2tag, f)
with open('models/bilstm_char/vocab.pkl', 'wb') as f:
    pickle.dump(vocab, f)
with open('models/bilstm_char/nertags.pkl', 'wb') as f:
    pickle.dump(nertags, f)
with open('models/bilstm_char/char_vocab.pkl', 'wb') as f:
    pickle.dump(char_vocab, f)
with open('models/bilstm_char/char_onehot.pkl', 'wb') as f:
    pickle.dump(char_onehot, f)
with open('models/bilstm_char/get_charvocab.pkl', 'wb') as f:
    pickle.dump(get_charvocab, f)

In [18]:
model.load_state_dict(torch.load('models/bilstm_char/trained_bilstm_model_state_dict.pth'))

# Put the model in evaluation mode
model.eval()

BiLSTM(
  (wordembed): Embedding(23626, 100)
  (for_charembed): forLSTM(
    (charembed): Embedding(88, 88)
    (lstm): LSTM(88, 25, batch_first=True, bidirectional=True)
  )
  (dropout): Dropout(p=0.5, inplace=False)
  (bilstm): LSTM(150, 100, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=200, out_features=18, bias=True)
)

In [19]:
model.eval()

BiLSTM(
  (wordembed): Embedding(23626, 100)
  (for_charembed): forLSTM(
    (charembed): Embedding(88, 88)
    (lstm): LSTM(88, 25, batch_first=True, bidirectional=True)
  )
  (dropout): Dropout(p=0.5, inplace=False)
  (bilstm): LSTM(150, 100, batch_first=True, bidirectional=True)
  (linear): Linear(in_features=200, out_features=10, bias=True)
)

In [20]:
def final_metrics(model,loader):
    y_predicted = []
    y_true = []
    with torch.no_grad():
        for step, (X,Xchar, Y, xlen, xlen_char) in enumerate(loader):
            Y = pack_padded_sequence(Y, xlen, batch_first=True, enforce_sorted=False)
            Y, _ = pad_packed_sequence(Y, batch_first=True)
            ypred = model(X.long().to(device), Xchar.to(device), xlen.to(device), xlen_char.to(device))#.permute(0, 2, 1)
            ypred = torch.argmax(ypred.to('cpu'), dim = 1)
            ypred = ypred.view(Y.shape[0], -1)
            y_predicted.append(ypred)
            y_true.append(Y)

    y_predicted_list = []
    y_true_list = []
    for i in range(len(y_predicted)):
        for j in range(y_predicted[i].shape[0]):
            sent_pred = []
            sent_true = []
            for x in range(y_predicted[i].shape[1]):
                sent_pred.append(id2tag[int(y_predicted[i][j, x])])
                sent_true.append(id2tag[int(y_true[i][j, x])])
            y_predicted_list.append(sent_pred)
            y_true_list.append(sent_true)
    
    return seq_f1_score(y_true_list, y_predicted_list), seq_accuracy_score(y_true_list, y_predicted_list), seq_classification_report(y_true_list, y_predicted_list, digits = 3)

In [21]:
#Test DATASET
char_vocab = get_charvocab(vocab)
Xtest, Ytest, x_testlengths, _, _ = load_data(testdatapath, buildvocab_tags=False, vocab = vocab, nertags = nertags)

Xtest_temp, Ytest_temp, x_testlengths_temp, testvocab, testnertags = load_data(testdatapath, buildvocab_tags=True)
wordid2word_charlevel_vocab_test, wordid2wordlen_test = make_id2word_charvocab(testvocab, char_vocab) # of the form {word:[1,2,3,4]}, {wordnum:wordlen}
#make char level train data for the char embeddings 
Xtest_char, xtestlength_char = load_char_level(Xtest_temp, wordid2word_charlevel_vocab_test, wordid2wordlen_test)
#finally make the dataloader for train
testdataset = TensorDataset(Xtest, Xtest_char, Ytest, x_testlengths, xtestlength_char)
loader_test = DataLoader(testdataset, batch_size= 1, shuffle=False)
test_f1_conll, test_acc_conll, test_classif_report = final_metrics(model, loader_test)

print("PERFORMANCE ON Test DATA")
print('MicroF1 = {}'.format(test_f1_conll))
print('Accuracy = {}'.format(test_acc_conll))
print('------------Classification Report-------------')
print(test_classif_report)

torch.Size([3683, 124]) torch.Size([3683, 124]) torch.Size([3683])
torch.Size([3683, 124]) torch.Size([3683, 124]) torch.Size([3683])
PERFORMANCE ON Test DATA
MicroF1 = 0.7276526162790697
Accuracy = 0.9377477767063109
------------Classification Report-------------
              precision    recall  f1-score   support

         LOC      0.897     0.761     0.824      1668
        MISC      0.740     0.679     0.708       702
         ORG      0.667     0.634     0.650      1661
         PER      0.701     0.745     0.722      1617

   micro avg      0.747     0.709     0.728      5648
   macro avg      0.751     0.705     0.726      5648
weighted avg      0.753     0.709     0.729      5648



In [22]:
#### deletE?

def out_predictions_char(model, loader, output_file):
    with open(output_file, 'w') as f:
        with torch.no_grad():
            for step, (X, Xchar, Y, xlen, xlen_char) in enumerate(loader):
                Y = pack_padded_sequence(Y, xlen, batch_first=True, enforce_sorted=False)
                Y, _ = pad_packed_sequence(Y, batch_first=True)
                ypred = model(X.long().to(device), Xchar.to(device), xlen.to(device), xlen_char.to(device))
                ypred = torch.argmax(ypred.to('cpu'), dim=1)
                ypred = ypred.view(Y.shape[0], -1)
                for i in range(len(ypred)):
                    for j in range(len(ypred[i])):
                        word = id2word[int(X[i, j])]
                        tag = id2tag[int(ypred[i, j])]
                        f.write(f"{word}\t{tag}\n")
                    f.write('\n')

# Test your character-based model and output predictions to a file
out_predictions_char(model, loader_test, 'predictions/gold_bilstm_random.txt')

In [23]:
def readBIO(path):
    ents = []
    curEnts = []
    for line in open(path):
        line = line.strip()
        if line == '':
            ents.append(curEnts)
            curEnts = []
        elif line[0] == '#' and len(line.split('\t')) == 1:
            continue
        else:
            curEnts.append(line.split('\t')[1])
    return ents

def toSpans(tags):
    spans = set()
    for beg in range(len(tags)):
        if tags[beg][0] == 'B':
            end = beg
            for end in range(beg+1, len(tags)):
                if tags[end][0] != 'I':
                    break
            spans.add(str(beg) + '-' + str(end) + ':' + tags[beg][2:])
            #print(end-beg)
    return spans

def getInstanceScores(predPath, goldPath):
    goldEnts = readBIO(goldPath)
    predEnts = readBIO(predPath)
    entScores = []
    tp = 0
    fp = 0
    fn = 0
    for goldEnt, predEnt in zip(goldEnts, predEnts):
        goldSpans = toSpans(goldEnt)
        predSpans = toSpans(predEnt)
        overlap = len(goldSpans.intersection(predSpans))
        tp += overlap
        fp += len(predSpans) - overlap
        fn += len(goldSpans) - overlap
        
    prec = 0.0 if tp+fp == 0 else tp/(tp+fp)
    rec = 0.0 if tp+fn == 0 else tp/(tp+fn)
    f1 = 0.0 if prec+rec == 0.0 else 2 * (prec * rec) / (prec + rec)
    return f1

In [24]:
pred = 'predictions/gold_bilstm_random.txt'
gold = 'data/gold.txt'

score = getInstanceScores(pred,gold)
print('Span-F1 score - char-RANDOM- biLSTM ', score)

Span-F1 score - char-RANDOM- biLSTM  0.7497194163860831


In [110]:
# SAVE MODEL AND DICTIONARIES

torch.save(model.state_dict(), 'models/bilstm_char/trained_bilstm_model_state_dict.pth')
torch.save(model, 'models/bilstm_char/trained_bilstm_model.pth')

with open('models/bilstm_char/id2word.pkl', 'wb') as f:
    pickle.dump(id2word, f)
with open('models/bilstm_char/id2tag.pkl', 'wb') as f:
    pickle.dump(id2tag, f)
with open('models/bilstm_char/vocab.pkl', 'wb') as f:
    pickle.dump(vocab, f)
with open('models/bilstm_char/nertags.pkl', 'wb') as f:
    pickle.dump(nertags, f)
with open('models/bilstm_char/char_vocab.pkl', 'wb') as f:
    pickle.dump(char_vocab, f)
with open('models/bilstm_char/char_onehot.pkl', 'wb') as f:
    pickle.dump(char_onehot, f)
with open('models/bilstm_char/get_charvocab.pkl', 'wb') as f:
    pickle.dump(get_charvocab, f)

#### NOISE INJECTION - altered dataset

In [25]:
noise_rates = {
    'capitalization_swap': [0.1, 0.15, 0.2, 0.25, 0.3], 
    'character_swap': [0.1, 0.15, 0.2, 0.25, 0.3],
    'character_removal': [0.1, 0.15, 0.2, 0.25, 0.3],
    'character_replacement': [0.1, 0.15, 0.2, 0.25, 0.3]
}

In [26]:
def test_load_data(datapath, buildvocab_tags= True, vocab = None, nertags = None):
    if(buildvocab_tags == True):
        all_words = []
        all_tags = []
        with open(datapath) as f:
            lines = f.readlines()
            sent_num = 0
            for line in lines: #1: so that the first blank line isn't taken into account
                if(line == "\n"):
                    sent_num+=1
                else:
                    line_sep = line.split('\t')
                    all_words.append(line_sep[0])
                    all_tags.append(line_sep[1][:-1])
                    
        words = list(set(all_words))
        tags = list(set(all_tags))

        vocab = {}
        vocab['<pad>'] = 0 # for padding input sequences
        vocab['<oov>'] = 1
        for i, word in enumerate(words):
            vocab[word] = i+2
            
        nertags = {}
        nertags['padtag'] = 0
        for i,nertag in enumerate(tags):
            nertags[nertag] = i+1

    train_sent = []
    train_tags = []
    with open(datapath) as f:
        lines = f.readlines()
        sent_num = 0
        sentence = []
        tag = []
        for line in lines: #1: so that the first blank line isn't taken into account
            if(line == "\n"):
                sent_num+=1
                train_sent.append(sentence)
                train_tags.append(tag)
                sentence = []
                tag = []
            else:
                line_sep = line.split("\t")
                if(line_sep[0] in vocab.keys()):
                    sentence.append(vocab[line_sep[0]])
                else:
                    sentence.append(vocab['<oov>'])  
                tag.append(nertags[line_sep[-1][:-1]])

    # padding the sentences at the end
    seq_maxlen = max(len(x) for x in train_sent)
    x_lengths = [len(x) for x in train_sent]
    Xtrain = []
    Ytrain = []
    for sent, tags in zip(train_sent, train_tags):
        length_toappend = seq_maxlen - len(sent)
        Xtrain.append(sent+[0]*length_toappend)
        Ytrain.append(tags+[0]*length_toappend)


    Xtrain = torch.Tensor(Xtrain)
    Ytrain = torch.Tensor(Ytrain)
    x_lengths = torch.Tensor(x_lengths)
    
    return Xtrain, Ytrain, x_lengths, vocab, nertags


In [27]:
def loader(testdatapath):
    Xtest, Ytest, x_testlengths, _, _ = test_load_data(testdatapath, buildvocab_tags=False, vocab = vocab, nertags = nertags)

    Xtest_temp, Ytest_temp, x_testlengths_temp, testvocab, testnertags = test_load_data(testdatapath, buildvocab_tags=True)
    wordid2word_charlevel_vocab_test, wordid2wordlen_test = make_id2word_charvocab(testvocab, char_vocab) # of the form {word:[1,2,3,4]}, {wordnum:wordlen}
    #make char level train data for the char embeddings 
    Xtest_char, xtestlength_char = load_char_level(Xtest_temp, wordid2word_charlevel_vocab_test, wordid2wordlen_test)
    #finally make the dataloader for train
    testdataset = TensorDataset(Xtest, Xtest_char, Ytest, x_testlengths, xtestlength_char)
    loader_test = DataLoader(testdataset, batch_size= 1, shuffle=False)
    return loader_test

In [28]:
def test_model(model, loader):
    y_predicted = []
    with torch.no_grad():
        for step, (X, Xchar, Y, xlen, xlen_char) in enumerate(loader):
            ypred = model(X.long().to(device), Xchar.to(device), xlen.to(device), xlen_char.to(device))#.permute(0, 2, 1)
            ypred = torch.argmax(ypred.to('cpu'), dim = 1)
            ypred = ypred.view(Y.shape[0], -1)
            y_predicted.append(ypred)

    y_predicted_list = []
    for i in range(len(y_predicted)):
        for j in range(y_predicted[i].shape[0]):
            sent_pred = []
            for x in range(y_predicted[i].shape[1]):
                sent_pred.append(id2tag[int(y_predicted[i][j, x])])
            y_predicted_list.append(sent_pred)
    return y_predicted_list

In [29]:
def writefile(testfilepath, outputfilepath, predictions):
    final_output = [] #list of lists which will finally be written to file
    with open(testfilepath) as f:
        lines = f.readlines()
        sentnum = 0 #to take care of the first blank line
        wordnum = 0
        for line in lines:
            if(line == '\n'):
                sentnum+=1
                wordnum = 0
                final_output.append(line)

            else:
                line_sep = line.split(sep = "\t")
                word = line_sep[0]
                prediction = predictions[sentnum][wordnum]
                final_output.append(f'{word}\t{prediction}\n')
                wordnum+=1
    #write the outputfilepath
    with open(outputfilepath, 'w+') as f:
        f.writelines(final_output)

In [30]:
for noise_type in noise_rates.keys():
    for rate in noise_rates[noise_type]:
        path = f'data/altered/{noise_type}_rate_{rate}.txt'
        my_loader = loader(path)
        predictions = test_model(model, my_loader)
        #print(predictions)
        writefile(path, f'predictions/altered/random/{noise_type}_rate_{rate}.txt',predictions)

In [31]:
types = []
rates = []
f1_score = []

for noise_type in noise_rates.keys():
    for rate in noise_rates[noise_type]:
        score = getInstanceScores(f'predictions/altered/random/{noise_type}_rate_{rate}.txt', 'data/gold.txt')
        
        # Append data to lists
        types.append(noise_type)
        rates.append(rate)
        f1_score.append(score)

results = {'Type': types, 'Rate': rates, 'F1 Score': f1_score}
df = pd.DataFrame(results)
print(df)

                     Type  Rate  F1 Score
0     capitalization_swap  0.10  0.681852
1     capitalization_swap  0.15  0.648644
2     capitalization_swap  0.20  0.617456
3     capitalization_swap  0.25  0.588180
4     capitalization_swap  0.30  0.560166
5          character_swap  0.10  0.720159
6          character_swap  0.15  0.534556
7          character_swap  0.20  0.361144
8          character_swap  0.25  0.290720
9          character_swap  0.30  0.274491
10      character_removal  0.10  0.716285
11      character_removal  0.15  0.501342
12      character_removal  0.20  0.288770
13      character_removal  0.25  0.221473
14      character_removal  0.30  0.224307
15  character_replacement  0.10  0.715989
16  character_replacement  0.15  0.498709
17  character_replacement  0.20  0.287100
18  character_replacement  0.25  0.220120
19  character_replacement  0.30  0.219637


In [32]:
df.to_csv('out/df_altered_random.csv', index=False)

In [33]:
nertags

{'padtag': 0,
 'B-LOC': 1,
 'I-MISC': 2,
 'B-PER': 3,
 'I-ORG': 4,
 'B-ORG': 5,
 'B-MISC': 6,
 'O': 7,
 'I-PER': 8,
 'I-LOC': 9}

### Altered_2

In [103]:
noise_rates = {
    'capitalization_swap': [0, 0.1, 0.2, 0.3, 0.4],  # Adjust the rates as needed
    'character_swap': [0, 0.05, 0.075, 0.1, 0.125, 0.15, 0.17, 0.2],
    'character_removal': [0, 0.05, 0.075, 0.1, 0.125, 0.15, 0.17, 0.2],
    'character_replacement': [0, 0.05, 0.075, 0.1, 0.125, 0.15, 0.17, 0.2]
}

In [104]:
for noise_type in noise_rates.keys():
    for rate in noise_rates[noise_type]:
        path = f'data/altered_2/{noise_type}_rate_{rate}.txt'
        my_loader = loader(path)
        predictions = test_model(model, my_loader)
        #print(predictions)
        writefile(path, f'predictions/altered_2/char/{noise_type}_rate_{rate}.txt',predictions)

In [105]:
types = []
rates = []
f1_score = []

for noise_type in noise_rates.keys():
    for rate in noise_rates[noise_type]:
        score = getInstanceScores(f'predictions/altered_2/char/{noise_type}_rate_{rate}.txt', 'data/gold.txt')
        
        # Append data to lists
        types.append(noise_type)
        rates.append(rate)
        f1_score.append(score)

results = {'Type': types, 'Rate': rates, 'F1 Score': f1_score}
df = pd.DataFrame(results)
print(df)

                     Type   Rate  F1 Score
0     capitalization_swap  0.000  0.745409
1     capitalization_swap  0.100  0.684309
2     capitalization_swap  0.200  0.622143
3     capitalization_swap  0.300  0.568596
4     capitalization_swap  0.400  0.515639
5          character_swap  0.000  0.745409
6          character_swap  0.050  0.745162
7          character_swap  0.075  0.743834
8          character_swap  0.100  0.702426
9          character_swap  0.125  0.630696
10         character_swap  0.150  0.565464
11         character_swap  0.170  0.502799
12         character_swap  0.200  0.440939
13      character_removal  0.000  0.745409
14      character_removal  0.050  0.745162
15      character_removal  0.075  0.743901
16      character_removal  0.100  0.698742
17      character_removal  0.125  0.616539
18      character_removal  0.150  0.539534
19      character_removal  0.170  0.464226
20      character_removal  0.200  0.399509
21  character_replacement  0.000  0.745409
22  charact

In [106]:
df.to_csv('out/df_altered_2_char.csv', index=False)